# Supervised Learning ASA

### Regression Model with Xgboost

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from xgboost import XGBRFRegressor

In [2]:
df = pd.read_csv('data_27-05-2021.csv')

In [3]:
df

,distance,aspect,delta_heading,delta_altitude,delta_velocity,wez_max_own2trk,wez_nez_own2trk,wez_max_trk2own,wez_nez_trk2own,vul_thr_bef_shot,...,rwr_aircraft_warning,hp_tgt_ofensivity,hp_thr_vulnerability,own_shot_phi_long,own_shot_phi_medium,own_shot_phi_short,enemy_shot_phi_long,enemy_shot_phi_medium,enemy_shot_phi_short,dca_index
0,87597.401454,156.659122,134.262134,1981.240840,-0.577486,35.762241,9.383960,31.925061,8.654041,0.675140,...,0.0,0.004381,0.004381,0,0,1,0,0,1,0.627679
1,88638.600816,156.580036,133.242155,-1979.063686,0.594680,31.882673,8.676122,35.587813,9.354046,0.698301,...,0.0,0.006144,0.006144,0,0,1,1,0,0,0.646715
2,88892.829708,150.982223,83.477963,-606.720943,-0.066980,19.331238,7.777890,29.767125,7.508884,0.577344,...,0.0,0.003989,0.003989,1,0,0,0,1,0,0.115633
3,88441.224917,155.621190,111.293288,151.014463,3.072693,28.309615,8.260176,31.454548,8.669155,0.954911,...,0.0,0.005535,0.005535,0,1,0,1,0,0,0.332373
4,35822.060007,165.901209,47.757196,143.428417,7.848019,10.170417,7.964732,-1.000000,-1.000000,0.954911,...,0.0,0.006054,0.000000,0,1,0,1,0,0,0.522806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9360,30734.453675,179.040462,33.188695,-223.458862,-17.099398,7.398858,6.527613,-1.000000,-1.000000,0.162927,...,0.0,0.006331,0.000000,0,0,1,1,0,0,0.674485
9361,72346.427927,130.523076,60.610186,2.323086,-0.028185,16.433046,6.061779,22.195293,5.196923,0.051978,...,0.0,0.004681,0.004681,1,0,0,1,0,0,0.298276
9362,16658.724021,134.425564,151.539888,-26.439097,0.115436,28.049797,7.049873,21.818230,7.000619,0.051978,...,0.0,0.970705,0.958764,1,0,0,1,0,0,0.478450
9363,69067.064285,144.954036,25.565549,1409.924388,-0.391399,9.548427,6.642626,-1.000000,-1.000000,0.387971,...,0.0,0.005659,0.000000,1,0,0,1,0,0,0.323546


In [4]:
# id_batch = 374 with 3729 executions
print('samples: ',len(df))

samples:  9365


### Train Test Split and Scaling

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X = df[['distance', 'hp_thr_vulnerability', 'hp_tgt_ofensivity', 'wez_max_own2trk', 'delta_velocity', 'wez_max_trk2own', 'delta_heading', 'wez_nez_trk2own', 'wez_nez_own2trk', 'aspect']]
y = df['dca_index']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state=101)

### XGBoost

XGBoost stands for Extreme Gradient Boosting. It is considered the "state-of-the-art” machine learning algorithm to deal with structured data.

#### Why use XGBoost?


* Speed and performance : Originally written in C++, it is comparatively faster than other ensemble classifiers.

* Core algorithm is parallelizable : Because the core XGBoost algorithm is parallelizable it can harness the power of multi-core computers. It is also parallelizable onto GPU’s and across networks of computers making it feasible to train on very large datasets as well.

* Consistently outperforms other algorithm methods : It has shown better performance on a variety of machine learning benchmark datasets.

* Wide variety of tuning parameters : XGBoost internally has parameters for cross-validation, regularization, user-defined objective functions, missing values, tree parameters, scikit-learn compatible API etc.

In [8]:
parameters={'n_estimators': [100, 250, 500, 750, 1000],
            'max_depth': [2, 3, 6],
            'learning_rate': [0.1, 0.01],
            'subsample': [0.6, 0.8],
            'colsample_bytree': [0.8, 1],
            'gamma': [0, 1],
            'min_child_weight': [1, 2]}

In [9]:
model = XGBRegressor(n_estimators = 300)

In [9]:
from sklearn.model_selection import GridSearchCV

model_grid = GridSearchCV(model, parameters, cv=3, verbose=1, scoring='r2', n_jobs = -1, return_train_score=True)
model_grid.fit(X,y)

Fitting 3 folds for each of 480 candidates, totalling 1440 fits


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Process LokyProcess-32:
Process LokyProcess-33:
Traceback (most recent call last):
  File "/home/jovyan/work/envs/asa/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/jovyan/work/envs/asa/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jovyan/work/envs/asa/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py", line 446, in _process_worker
    _process_reference_size = _get_memory_usage(pid, force_gc=True)
  File "/home/jovyan/work/envs/asa/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py", line 116, in _get_memory_usage
    gc.collect()
KeyboardInterrupt
Process LokyProcess-30:
Traceback (most recent call last):
  File "/home/jovyan/work/envs/asa/lib/python3.9/site-packages/joblib/_parallel_backends.py", line 595, in __call__
 

KeyboardInterrupt: 

In [ ]:
# best parameters
model_grid.best_params_

In [ ]:
model.fit(X_train,y_train, eval_metric=['rmse'], eval_set = [(X_train, y_train), (X_test, y_test)], verbose=0, 
          early_stopping_rounds=300)

In [ ]:
from matplotlib import pyplot

 # retrieve performance metrics
results = model.evals_result()
epochs = len(results['validation_0']['rmse'])
x_axis = range(0, epochs)

# plot rmse
fig, ax = pyplot.subplots(figsize=(15,8))
ax.plot(x_axis, results['validation_0']['rmse'], label='Train')
ax.plot(x_axis, results['validation_1']['rmse'], label='Test')
ax.legend()
pyplot.ylabel('RMSE')
pyplot.title('XGBoost RMSE');

In [ ]:
# make predictions for test data
predictions = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score,r2_score

# Calculate the Explained Variance Score
print("Explained Variance Score: {:.3f}".format(explained_variance_score(predictions,y_test)))
# Calculate the R2
print("R2: {:.3f}".format(r2_score(y_test,predictions)))
# Calculate the Mean Abosolute Error
print("MAE: {:.3f}".format(mean_absolute_error(y_test,predictions)))
# Calculate the Mean Squared Error
print("MSE: {:.3f}".format(mean_squared_error(y_test,predictions)))
# Calculate the Root Mean Squared Error
print("RMSE: {:.3f}".format(np.sqrt(mean_squared_error(y_test,predictions))))

In [ ]:
df[['air_superiority_index']].describe().T

In [ ]:
# Calculate the Mean and Standard Deviation of the Air Superiority Index
print("Mean of ASI: {:.3f}".format(y.mean()))
print("STD of ASI: {:.3f}".format(y.std()))

In [ ]:
plt.figure(figsize=(20,8))

# Our predictions
plt.scatter(y_test,predictions)

# Perfect predictions
plt.plot(y_test,y_test,'r');

In [ ]:
from xgboost import plot_importance
plt.rcParams["figure.figsize"] = (15,8)


_ = plot_importance(model, height=0.9)

### Explore Number of Trees

In [ ]:
# get a list of models to evaluate
def get_models():
    models = dict()
    # define the number of trees to consider
    n_trees = [10, 50, 100, 500, 1000, 5000]
    for v in n_trees:
        models[str(v)] = XGBRFClassifier(n_estimators=v, subsample=0.9, colsample_bynode=0.2)
    return models
 
# evaluate a give model using cross-validation
def evaluate_model(model, X, y):
    # define the model evaluation procedure
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    # evaluate the model
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    return scores

# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    # evaluate the model and collect the results
    scores = evaluate_model(model, X, y)
    # store the results
    results.append(scores)
    names.append(name)
    # summarize performance along the way
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

### Saving the model

In [ ]:
import joblib

# save model to file
joblib.dump(model, 'asa_model_xgboost.dat');

### Evaluation the Learning Curve

In [ ]:
from sklearn.model_selection import learning_curve

# Create CV training and test scores for various training set sizes
train_sizes, train_scores, test_scores = learning_curve(model,
                                           X, y, cv=5, scoring='r2', n_jobs=-1,
                                           # 50 different sizes of the training set
                                           train_sizes=np.linspace(0.01, 1.0, 50))

In [ ]:
# Create means and standard deviations of training set scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

# Create means and standard deviations of test set scores
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Draw lines
plt.subplots(1, figsize=(12,12))
plt.plot(train_sizes, train_mean, '--', color="#111111",  label="Training score")
plt.plot(train_sizes, test_mean, color="#111111", label="Cross-validation score")

# Draw bands
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="#DDDDDD")
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color="#DDDDDD")

# Create plot
plt.title("Learning Curve")
plt.xlabel("Training Set Size")
plt.ylabel("R2 Score")
plt.legend(loc="best")
plt.tight_layout();

### Deployment

In [ ]:
# Taking a sample
X_sample = df.drop('air_superiority_index', axis=1).iloc[0:1]
y_sample = df['air_superiority_index'].iloc[0:1]

In [ ]:
X_sample.shape

In [ ]:
X_sample

In [ ]:
y_sample

In [ ]:
import joblib

#load model from file
loaded_model = joblib.load('asa_model_xgboost.dat')

In [ ]:
# make predictions for test data
y_pred = loaded_model.predict(X_sample)

In [ ]:
# predicted Value
y_pred[0]

In [ ]:
# true Valeu
df['air_superiority_index'].iloc[0]